In [2]:
import torch  # Add this import

from transformers import AutoTokenizer, AutoModelForCausalLM
import os
import json
import random
from datetime import datetime
from tqdm import tqdm
# Load your fine-tuned model
model_name = "./fine"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)


`low_cpu_mem_usage` was None, now default to True since model is quantized.


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 3072, padding_idx=128004)
    (layers): ModuleList(
      (0-27): 28 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=3072, out_features=3072, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=3072, out_features=16, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=16, out_features=3072, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=3072, out_features=1024, bias=False)
            (lora_dropout)

In [3]:
# model, tokenizer, device = load_llama_model()

# Define cognitive distortions
cognitive_distortions = {
   "All or Nothing Thinking": "I view a situation, a person or an event in “either-or” terms, fitting them into only two extreme categories instead of on a continuum.",
    "Fortune Telling": "I predict the future in negative terms and believe that what will happen will be so awful that I will not be able to stand it.",
    "Emotional Reasoning": "I believe my emotions reflect reality and let them guide my attitudes and judgments.",
    "Labeling/Global Labeling": "I put a fixed, usually negative, global label on myself or others.",
    "Mental Filter": "I pay attention to one or a few details and fail to see the whole picture.",
    "Mind Reading": "I believe that I know the thoughts or intentions of others (or that they know my thoughts or intentions) without sufficient evidence.",
    "Overgeneralization": "I take isolated negative cases and generalize them, using words like “always,” “never,” “whole,” “entire,” etc.",
    "Personalization": "I assume that others’ behaviors and external events concern myself without considering other plausible explanations.",
    "Should Statements": "I tell myself that events, people’s behaviors, and my own attitudes “should” be the way I expected, not as they are.",
    "Blaming": "I direct blame to others for my negative feelings or take responsibility for others' behaviors and attitudes.",
    "What if?": "I keep asking questions like “what if something happens?” focusing on negative outcomes.",
    "Discounting the Positive": "I disqualify positive experiences or events, insisting that they don’t count.",
    "Magnification/Minimization": "I emphasize the negatives or downplay positives in myself, others, or situations.",
    "Jumping to Conclusions": "I draw conclusions from little or no confirmatory evidence.",
    "Unfair Comparisons": "I compare myself with others who seem better and place myself at a disadvantage."
}


# Variants of writing style for diversity
tones = ["angry", "calm", "frustrated", "sarcastic", "venting", "blunt", "sad", "anxious", "direct"]
structures = ["structured", "scattered", "nuanced", "blunt", "sarcastic"]
intensities = ["mild", "intense", "slightly annoyed", "highly emotional", "casual"]
use_of_language = ["use cuss words", "avoid cuss words", "use formal language", "use casual language"]

system_message = """
You are an expert assistant specializing in identifying and generating text examples that reflect common cognitive distortions.
Your task is to generate writing prompts and then create text from the perspective of people experiencing specific cognitive distortions without being aware of it.
Each time you are asked to generate text, remember that the person should not be aware of their cognitive distortion.
Make sure the situation, writing style, and personality of the person are different for each response.
The final text should demonstrate the cognitive distortion in a subtle, human, realistic way.
Make responses realistic, expressive, and aligned with the triggering event.
"""



In [4]:
import re
def generate_response(text: str, count: int) -> str:
    inputs = tokenizer(text, return_tensors="pt").to(device)

    # Generate the response
    outputs = model.generate(
        **inputs,
        max_new_tokens=count,
        temperature=0.6,
        top_p=0.9,
        repetition_penalty=1.1,
        do_sample=True,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

    # Decode the generated output
    response_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return response_text


def clean_text(text):
    # Remove actual newline and carriage return characters.
    text = text.replace('\n', ' ').replace('\r', ' ')
    # Remove literal escape sequences for newline (e.g., "\n")
    text = re.sub(r'\\n', ' ', text)
    # Remove Unicode escape sequences (e.g., "\u1234")
    text = re.sub(r'\\u[0-9A-Fa-f]{4}', '', text)
    # Remove any remaining backslashes.
    text = re.sub(r'\\', '', text)
    # Optionally, remove extra whitespace.
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

In [5]:
import glob


In [ ]:
# Load pre-generated trigger events from multiple JSON files in the data directory
trigger_events_dir = "/home/g4/Mindwell/DataGenerationCBTApproach/trigger_events_nemo_3_nostructure/new_800"
# Adjust the glob pattern if necessary; here we load all JSON files.
trigger_events_files = glob.glob(os.path.join(trigger_events_dir, "*.json"))

trigger_events = []
for file in trigger_events_files:
    with open(file, "r") as f:
        data = json.load(f)
        # Assuming each file contains a list of trigger events
        if data != []:
            trigger_events.append(data)

# Generate subjective assessments based on trigger events
output_dir = "/home/g4/Mindwell/DataGenerationCBTApproach/data_nemo_fine_nostructure/new_800_data"
os.makedirs(output_dir, exist_ok=True)

for event_file in tqdm(trigger_events, desc="Generating Assessments"):
    generated_data = []
    for event in event_file:
        distortion = random.choice(list(cognitive_distortions.keys()))  # Randomly assign a cognitive distortion
        user_message = (
            f"Based on the following event, generate a subjective reaction reflecting the cognitive distortion '{distortion}':\n"
            f"Triggering Event: {event['Generated Scenario']}\n\n"
            f"The response should reflect the {distortion, cognitive_distortions[distortion]}' subtly, "
            f"without explicitly stating the distortion. "
            f"Make sure the person seems unaware of their cognitive bias.\n\n"
            f"Output only the example response, and do not include the prompt or any extra commentary"
            "### Response Start\n\n"
        )
    
        try:
            user_message += "\n\n### Response Start\n\n"
            generated_assessment = generate_response(user_message, 300).strip()

            if "### Response Start" in generated_assessment:
                generated_assessment = generated_assessment.split("### Response Start")[-1].strip()
            
          
            if "### Response End" in generated_assessment:
                generated_assessment = generated_assessment.split("### Response End")[0].strip()

            generated_assessment = clean_text(generated_assessment)
        
            generated_data.append({
                "Triggering Event": event["Generated Scenario"],
                "Sex": event["Sex"],
                "Age": event["Age"],
                "Occupation": event["Occupation"],
                "Relationship Status": event["Relationship Status"],
                "Negative Emotion": event["Negative Emotion"],
                "Cognitive Distortion": distortion,
                "Generated Assessment": generated_assessment
            })
        except Exception as e:
            print(f"Error generating assessment: {e}")

        exit()

    # Save the generated data
    print(generated_assessment)

    timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
    file_name = f"{output_dir}/{timestamp}_subjective_assessments.json"
    with open(file_name, 'w', encoding='utf-8') as f:
        json.dump(generated_data, f, indent=4, ensure_ascii=False)

    print(f"Saved {len(generated_data)} subjective assessments to {file_name}")

Generating Assessments:   0%|          | 1/800 [02:08<28:35:27, 128.82s/it]

This is so typical of you. You're always getting in the way, slowing everything down. Why don't you just take your old age and retire? No one needs someone like you around anymore. Just leave it alone and let us handle it. You've been doing this job for years, why can't you figure out how to fix it? You really are so annoying. Can't you see we're busy? What's wrong with you? Do you want to be fired? If you keep acting like this, you'll get fired for sure. You need to stop being such a pain in the ass. Maybe if you were younger, you wouldn't be so slow. Now, stop wasting everyone's time and go home. We don't have all day. What's taking so long? Why doesn't it work? Can't you even manage to fix a simple cash register? You must be so incompetent. Your lack of skills is embarrassing. You probably won't last much longer anyway. You're just holding back the rest of us who are smarter and faster. Everyone knows you're struggling to keep up these days. Don't pretend like you can still do the j

Generating Assessments:   0%|          | 2/800 [04:10<27:38:36, 124.71s/it]

She thought about how much he'd miss wearing those suits. They reminded him of a successful career before life got complicated again. It was probably just a mistake for her to let them go to waste. She wondered if she could somehow make him feel better by cleaning them, putting them back in the closet where he'd left them. That way, she could tell herself she was doing something thoughtful for him, even though it wouldn't change anything.
Saved 10 subjective assessments to /home/g4/Mindwell/DataGenerationCBTApproach/data_nemo_fine_nostructure/new_800_data/2025-05-03_20-33-43_subjective_assessments.json


Generating Assessments:   0%|          | 3/800 [06:03<26:21:41, 119.07s/it]

...

Generating Assessments: 100%|█████████▉| 799/800 [28:05:24<02:07, 127.01s/it]

The chef seemed really grossed out. I guess he was expecting fresh food. It’s weird how he didn’t notice that before he started cooking, though. It would have been obvious if he looked closely.
Saved 10 subjective assessments to /home/g4/Mindwell/DataGenerationCBTApproach/data_nemo_fine_nostructure/new_800_data/2025-05-05_00-34-57_subjective_assessments.json


Generating Assessments: 100%|██████████| 800/800 [28:07:33<00:00, 126.57s/it]

**Subjective Reaction** She was furious when she opened the email that rejected her paper, "A Lifetime of Research." The title, which included her late husband's name, seemed to mock her now, as if it were a cruel joke. This made her angry enough to lash out at her editor, whom she had trusted to help her through this difficult time. She felt betrayed by her editor's actions, especially since he had been so supportive during the submission process. It seemed like he was trying to humiliate her with his rejection letter, making her wonder if he was playing some kind of twisted game. When she confronted him about her anger, she found herself saying things that didn't feel like them. She accused him of being insensitive and heartless, implying that he must have done it on purpose to make her feel worse. As she ranted, she realized how much she was blaming him for her pain. It wasn't just about the rejection; it was about the way she felt - hurt, frustrated, and like she'd failed. Her ange

: 

In [ ]:
# Load pre-generated trigger events from multiple JSON files in the data directory
trigger_events_dir = "/home/g4/Mindwell/DataGenerationCBTApproach/trigger_events_nemo_3_nostructure/new_800"
# Adjust the glob pattern if necessary; here we load all JSON files.
trigger_events_files = glob.glob(os.path.join(trigger_events_dir, "*.json"))

trigger_events = []
for file in trigger_events_files:
    with open(file, "r") as f:
        data = json.load(f)
        # Assuming each file contains a list of trigger events
        if data != []:
            trigger_events.append(data)

# Generate subjective assessments based on trigger events
output_dir = "/home/g4/Mindwell/DataGenerationCBTApproach/data_nemo_fine_structure/new_800_data"
os.makedirs(output_dir, exist_ok=True)

for event_file in tqdm(trigger_events, desc="Generating Assessments"):
    generated_data = []
    for event in event_file:
        distortion = random.choice(list(cognitive_distortions.keys()))  # Randomly assign a cognitive distortion
    
        user_message = (
            f"Based on the following event, generate a subjective reaction reflecting the cognitive distortion '{distortion}':\n"
            f"Triggering Event: {event['Generated Scenario']}\n\n"
            f"The response should reflect '{cognitive_distortions[distortion]}' subtly, "
            f"without explicitly stating the distortion. "
            f"Make sure the person seems unaware of their cognitive bias.\n\n"
            f"Use a {random.choice(tones)} tone, {random.choice(structures)} structure, and {random.choice(intensities)} intensity. "
            f"{random.choice(use_of_language)} in the response."
            f"Output only the example response, and do not include the prompt or any extra commentary"
            "### Response Start\n\n"
        )
    
        try:
            user_message += "\n\n### Response Start\n\n"
            generated_assessment = generate_response(user_message, 300).strip()

            if "### Response Start" in generated_assessment:
                generated_assessment = generated_assessment.split("### Response Start")[-1].strip()
            
          
            if "### Response End" in generated_assessment:
                generated_assessment = generated_assessment.split("### Response End")[0].strip()

            generated_assessment = clean_text(generated_assessment)
        
            generated_data.append({
                "Triggering Event": event["Generated Scenario"],
                "Sex": event["Sex"],
                "Age": event["Age"],
                "Occupation": event["Occupation"],
                "Relationship Status": event["Relationship Status"],
                "Negative Emotion": event["Negative Emotion"],
                "Cognitive Distortion": distortion,
                "Generated Assessment": generated_assessment
            })
        except Exception as e:
            print(f"Error generating assessment: {e}")

        exit()

    # Save the generated data
    print(generated_assessment)

    timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
    file_name = f"{output_dir}/{timestamp}_subjective_assessments.json"
    with open(file_name, 'w', encoding='utf-8') as f:
        json.dump(generated_data, f, indent=4, ensure_ascii=False)

    print(f"Saved {len(generated_data)} subjective assessments to {file_name}")

Generating Assessments:  95%|█████████▍| 757/800 [27:30:10<1:34:44, 132.21s/it]

Ugh, I felt so fucking pissed off when that happened. It just didn't seem fair—especially since we'd worked on that project together. But then again, who am I kidding? I'm always the one who does all the heavy lifting anyway. If he hadn't been around, that paper wouldn't have even gotten published. And yet, everyone's acting like he's some kind of genius overnight. What really gets me is how he's just standing there with this smug look on his face, basking in the praise like he deserves it. Meanwhile, no one gives a shit about me anymore. I mean, I know I've been quiet lately, but at least I wasn't the one getting all the attention. The way people are treating him now—it's just ridiculous. Like, what’s the point of even doing anything if you're not going to get recognized? I swear, the world has gone mad. Everyone's too busy patting themselves on the back and praising each other for whatever little achievements they think they deserve. And don’t even get me started on how he’s been tal

Generating Assessments:  95%|█████████▍| 758/800 [27:32:28<1:33:48, 134.00s/it]

Oh my god I'm fucking dying! What if she's dead what if she's been kidnapped what if she's lost her mind what if she's a ghost what if she's just some crazy ass bitch who came knocking because she wanted to scare the shit out of me what if that's exactly what she did what if she was going to kill me right there in my doorway what if it wasn't even a doorbell it was some kind of trap some kind of sick joke some kind of twisted game what if she was playing some kind of fucked up psychological warfare thing what if she wanted to break me what if she wanted to make me lose it what if she wanted to turn me into some kind of psycho killer what if I was already a psycho killer and she was just trying to bring me back to life what if I had killed her what if I had gone off the deep end and snapped what if I couldn't control myself anymore what if I became a monster what if I started killing people left and right what if I became some kind of serial killer what if I went insane what if I became

Generating Assessments:  95%|█████████▍| 759/800 [27:34:45<1:32:17, 135.06s/it]

...

Generating Assessments: 100%|█████████▉| 799/800 [29:04:59<02:10, 130.87s/it]

Oh my god, I can't even right now. It's like he just got the worst luck ever, you know? I mean, who finds a rotting fish like that in the freezer? And to think he had to touch it before he realized what it was! Just thinking about how gross it must've been makes me want to gag. It's like the universe decided to play a cruel joke on him right when he needed it most. I'm sure he'll have nightmares over this one... But honestly, I feel bad for him. I mean, who gets handed such a terrible surprise? The guy's already dealing with divorce; does he really need this kind of stress too? It's crazy how things like this can turn our days upside down. I can only imagine how much worse his day is going to get from here. Poor guy... **Edit:** I didn’t realize people would take this seriously, so thank you all for your thoughtful responses. I understand now that it’s hard to recognize these biases ourselves, especially when we’re in the middle of experiencing them. This has helped me become more awar

Generating Assessments: 100%|██████████| 800/800 [29:07:07<00:00, 131.03s/it]

It's unfortunate that she has been rejected again. I don't know why she is so obsessed with this research. It was all about her husband, after all. They had nothing to do with my life. I'm still waiting for my paper to be reviewed. But I guess it's no surprise that they rejected hers. After all, she did take credit for his work. And now, because of this, she gets to go back to her university as a professor? That's just unfair. I mean, who does she think she is? My research is better than hers, but nobody will ever acknowledge it. Nobody cares about my work. I have wasted years trying to publish papers, only to be ignored by everyone. Meanwhile, she can just coast through academia because of her late husband. I wish someone would tell me how much I am important too. Maybe then people would recognize my work and give me the recognition I deserve.
Saved 10 subjective assessments to /home/g4/Mindwell/DataGenerationCBTApproach/data_nemo_fine_structure/new_800_data/2025-05-06_17-07-10_subjec

: 